# Data Collection with PushShift API

Two loops searching over the two subreddits. The base epoch_time is 10AM PST 2021/07/30. Searching backwards, submissions are taken 100 at a time with newest duplicates removed. Add a `full_text` column corresponding to a combination of the title and self_text. 

In [1]:
#import necessary libraries
import pandas as pd
import numpy as np
import datetime as dt
import time
import requests

In [ ]:
#10AM PST 2021/07/30
epoch_time = 1627664412
#empty df to append to as we go along
dj_all = pd.DataFrame()
#which columns we care for
column_list = ['title', 'selftext', 'subreddit','created_utc', 'author', 'num_comments', 'score', 'is_self']
#loop until we have at least 6000 "unique" submissions
while dj_all.shape[0] < 6000:
    #dadjoke url, updating epoch as we go along
    url_dj = f"https://api.pushshift.io/reddit/search/submission?subreddit=dadjokes&size=100&before={epoch_time}"
    res_dj = requests.get(url_dj)
    if res_dj.status_code != 200:
        #in case we get a time out error, try again after 5s 3 times then fully break
        print("ruh roh... sleep and retry")
        failed +=1
        if failed >=3:
            print('Big ruh roh')
            break
        time.sleep(5)
        continue
    failed = 0

    #Combine new entries to ones we have, drop newest duplicates, wait 5s
    dj_temp = pd.DataFrame(res_dj.json()['data'])[column_list]
    epoch_time = dj_temp['created_utc'].min()
    time.sleep(5)
    dj_all = pd.concat([dj_all,dj_temp])
    dj_all.drop_duplicates('selftext',keep='last',inplace=True)
    
    print(epoch_time,dj_all.shape[0])

In [ ]:
#Follows the same method as above, just with antijokes instead
epoch_time = 1627664412
aj_all = pd.DataFrame()

failed = 0
while aj_all.shape[0] < 6000:
    url_aj = f"https://api.pushshift.io/reddit/search/submission?subreddit=AntiJokes&size=100&before={epoch_time}"
    res_aj = requests.get(url_aj)
    if res_aj.status_code != 200:
        print("ruh roh... sleep and retry")
        failed +=1
        if failed >=3:
            print('Big ruh roh')
            break
        time.sleep(5)
        continue
    failed = 0
    aj_temp = pd.DataFrame(res_aj.json()['data'])[column_list]
    #print(dj_temp.shape)
    epoch_time = aj_temp['created_utc'].min()
    time.sleep(5)
    aj_all = pd.concat([aj_all,aj_temp])
    #print(dj_all.shape)
    aj_all.drop_duplicates('selftext',keep='last',inplace=True)
    
    print(epoch_time,aj_all.shape[0])


In [ ]:
#Combine the two into 1 dataframe.
jokes_all = pd.concat([dj_all,aj_all])
#Add full_text as combined title and self
jokes_all['full_text'] = jokes_all['title'] + " " + jokes_all['selftext']
jokes_all = jokes_all[(jokes_all['selftext'] != '[deleted]') & (jokes_all['selftext'] != '[removed]')]
#drop nulls,think I randomly had 1
jokes_all.dropna(inplace=True)
#Save
# jokes_all.to_csv('../data/jokes.csv')